In [73]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error

# Load the data
ground_truth_path = '/home/zihan/bounding_box/annotated_gaze/002_tobii_afunc_appear3.csv.csv'
predictions_path = '/home/zihan/bounding_box/annotated_gaze_webgazer/002_webgazer_tobiitime_afunc_appear3.csv.csv'
ground_truth = pd.read_csv(ground_truth_path)
predictions = pd.read_csv(predictions_path)

# Assuming the geometry column is correctly positioned
ground_truth = ground_truth.iloc[:, :-3]  # adjust the slice if necessary
predictions = predictions.iloc[:, :-3]    # adjust the slice if necessary

# Melt the DataFrames to long format
ground_truth = pd.melt(ground_truth, id_vars=ground_truth.columns[:18], value_vars=ground_truth.columns[18:], var_name='token_gt', value_name='Value')
predictions = pd.melt(predictions, id_vars=predictions.columns[:7], value_vars=predictions.columns[7:], var_name='token_pred', value_name='Value')

# Function to extract coordinates from geometry column
def extract_coordinates(df):
    coords = df['geometry'].str.extract(r'POINT \(([-\d\.]+) ([-\d\.]+)\)')
    df['x'], df['y'] = coords[0].astype(float), coords[1].astype(float)
    return df.drop(columns=['geometry'])

# Apply the function
ground_truth = extract_coordinates(ground_truth)
predictions = extract_coordinates(predictions)

# Drop missing values and reset index
ground_truth.dropna(inplace=True)
ground_truth.reset_index(drop=True, inplace=True)
predictions.dropna(inplace=True)
predictions.reset_index(drop=True, inplace=True)

# Normalize features using StandardScaler
scaler = StandardScaler()
ground_truth[['x', 'y']] = scaler.fit_transform(ground_truth[['x', 'y']])
predictions[['x', 'y']] = scaler.transform(predictions[['x', 'y']])

# Function to create sequences for LSTM input
def create_sequences(data, sequence_length=10):
    xs, ys = [], []
    for i in range(len(data) - sequence_length):
        xs.append(data.iloc[i:(i + sequence_length)][['x', 'y']].values)
        ys.append(data.iloc[i + sequence_length][['x', 'y']].values)
    return np.array(xs), np.array(ys)

# Prepare training datasets
sequence_length = 10
X, y = create_sequences(ground_truth, sequence_length)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to float32 for model compatibility
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_val = X_val.astype(np.float32)
y_val = y_val.astype(np.float32)

# Define a simpler LSTM model with stronger regularization
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, kernel_regularizer='l2'),
    Dropout(0.5),
    LSTM(32, kernel_regularizer='l2'),
    Dropout(0.5),
    Dense(32, activation='relu', kernel_regularizer='l2'),
    Dropout(0.5),
    Dense(2)  # Outputs x and y
])
model.compile(optimizer='adam', loss='mse')

# Early stopping and learning rate reduction on plateau
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Train the model with validation data
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr])

# Prepare test data
X_test, y_test = create_sequences(predictions, sequence_length)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

# Predict using the LSTM model
predicted_values = model.predict(X_test)

# Adjust the predictions DataFrame to match the sequence length reduction
predictions_adjusted = predictions.iloc[sequence_length:sequence_length + len(predicted_values)].reset_index(drop=True)
predictions_adjusted['predicted_x'], predictions_adjusted['predicted_y'] = predicted_values[:, 0], predicted_values[:, 1]

# Add ground truth tokens to the predictions_adjusted DataFrame for comparison
predictions_adjusted['ground_truth_token'] = ground_truth['token_gt'].iloc[sequence_length:sequence_length + len(predicted_values)].values

# Save the corrected predictions
predictions_adjusted.to_csv('corrected_predictions_LSTM.csv', index=False)

# Evaluate the model
mse = mean_squared_error(y_test, predicted_values)
print(f'Mean Squared Error: {mse}')


Epoch 1/50


/home/zihan/anaconda3/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 280ms/step - loss: 1.9404 - val_loss: 1.9172 - learning_rate: 0.0010
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 1.8895 - val_loss: 1.8696 - learning_rate: 0.0010
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 1.8397 - val_loss: 1.8210 - learning_rate: 0.0010
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 1.8097 - val_loss: 1.7755 - learning_rate: 0.0010
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 1.7536 - val_loss: 1.7333 - learning_rate: 0.0010
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 1.6973 - val_loss: 1.6931 - learning_rate: 0.0010
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 1.6946 - val_loss: 1.6549 - learning_rate: 0.0010
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 1.6497 - val_loss: 1.6184 - learning_rate: 0.0010
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 1.6126 - val_loss: 1.5850 - learning_rate: 0.0010
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/st

In [70]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Load the data
ground_truth_path = '/home/zihan/bounding_box/annotated_gaze/002_tobii_afunc_appear3.csv.csv'
predictions_path = '/home/zihan/bounding_box/annotated_gaze_webgazer/002_webgazer_tobiitime_afunc_appear3.csv.csv'
ground_truth = pd.read_csv(ground_truth_path)
predictions = pd.read_csv(predictions_path)

# Assuming the geometry column is correctly positioned
ground_truth = ground_truth.iloc[:, :-3]  # adjust the slice if necessary
predictions = predictions.iloc[:, :-3]    # adjust the slice if necessary

# Melt the DataFrames to long format
ground_truth = pd.melt(ground_truth, id_vars=ground_truth.columns[:18], value_vars=ground_truth.columns[18:], var_name='token_gt', value_name='Value')
predictions = pd.melt(predictions, id_vars=predictions.columns[:7], value_vars=predictions.columns[7:], var_name='token_pred', value_name='Value')

# Function to extract coordinates from geometry column
def extract_coordinates(df):
    coords = df['geometry'].str.extract(r'POINT \(([-\d\.]+) ([-\d\.]+)\)')
    df['x'], df['y'] = coords[0].astype(float), coords[1].astype(float)
    return df.drop(columns=['geometry'])

# Apply the function
ground_truth = extract_coordinates(ground_truth)
predictions = extract_coordinates(predictions)

# Drop missing values and reset index
ground_truth.dropna(inplace=True)
ground_truth.reset_index(drop=True, inplace=True)
predictions.dropna(inplace=True)
predictions.reset_index(drop=True, inplace=True)

# Normalize features using StandardScaler
scaler = StandardScaler()
ground_truth[['x', 'y']] = scaler.fit_transform(ground_truth[['x', 'y']])
predictions[['x', 'y']] = scaler.transform(predictions[['x', 'y']])

# Prepare the features and targets
sequence_length = 10
def create_sequences(data, target, sequence_length=10):
    xs, ys = [], []
    for i in range(len(data) - sequence_length):
        xs.append(data.iloc[i:(i + sequence_length)][['x', 'y']].values)
        ys.append(target.iloc[i + sequence_length][['x', 'y']].values)
    return np.array(xs), np.array(ys)

X_train, y_train = create_sequences(ground_truth, ground_truth, sequence_length)
X_test, y_test = create_sequences(predictions, ground_truth, sequence_length)  # Use ground_truth for y_test to simulate correction

# Reshape data to fit the CNN input
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2]))

# Define and compile the CNN model
model = Sequential([
    Conv1D(64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Conv1D(32, kernel_size=2, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(2)  # Outputs x and y
])
model.compile(optimizer='adam', loss='mse')

# Convert data to float32 for model compatibility
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

# Predict using the CNN model
predicted_values = model.predict(X_test)

# Adjust the predictions DataFrame to match the sequence length reduction
predictions_adjusted = predictions.iloc[sequence_length:sequence_length + len(predicted_values)].reset_index(drop=True)
predictions_adjusted['predicted_x'], predictions_adjusted['predicted_y'] = predicted_values[:, 0], predicted_values[:, 1]

# Function to find nearest ground truth token based on Euclidean distance
def find_nearest_ground_truth(predicted_point, ground_truth):
    distances = np.sqrt((ground_truth['x'] - predicted_point[0])**2 + (ground_truth['y'] - predicted_point[1])**2)
    min_index = np.argmin(distances)
    return ground_truth.iloc[min_index]['token_gt']

# Map predictions to nearest ground truth tokens
predictions_adjusted['corrected_token'] = predictions_adjusted.apply(lambda row: find_nearest_ground_truth([row['predicted_x'], row['predicted_y']], ground_truth), axis=1)

# Add ground truth tokens to the predictions_adjusted DataFrame for comparison
predictions_adjusted['ground_truth_token'] = ground_truth['token_gt'].iloc[sequence_length:sequence_length + len(predicted_values)].values

# Save the corrected predictions
predictions_adjusted.to_csv('corrected_predictions_CNNs.csv', index=False)

# Optionally evaluate the model
mse = mean_squared_error(y_test, predicted_values)
print(f'Mean Squared Error: {mse}')


Epoch 1/20


/home/zihan/anaconda3/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.6586 - val_loss: 1.2092
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.6108 - val_loss: 1.1912
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.6363 - val_loss: 1.1797
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.6298 - val_loss: 1.1728
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.6041 - val_loss: 1.1675
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.5686 - val_loss: 1.1618
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.5672 - val_loss: 1.1568
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.5755 - val_loss: 1.1509
Epoch 9/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.5108 - val_loss: 1.1470
Epoch 10/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.5780 - val_loss: 1.1448
Epoch 11/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.4625 - val_loss: 1.1414
Epoch 12/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.4364 - val_loss: 1.1382
Epoch 13/20
